In [ ]:
import numpy as np
import pandas as pd
import glob
import os
 
import tensorflow as tf
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Input, GRU, LSTM

from keras.callbacks import EarlyStopping, CSVLogger, TerminateOnNaN
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA


In [ ]:
from weightedValues import weightValues

In [ ]:
def OpenJsontoArr(path):
    file = open(path)           #"X:\\KI Praktikum\\validate_Data\\3darray_x_cv.json"
    x_3d = json.load(file)
    file.close()
    x_3d = np.asarray(x_3d)
    return x_3d

In [ ]:
def get_lstm_model(layers=[5, 5, 5], dropout=0, activation = "relu", input_shape = (1, 1), loss = "mean_squared_error"):
    
    model = Sequential()
    model.add(Input(input_shape))
    
    for i in range(len(layers)):
        if i == len(layers)-1:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=False))
        else:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=True))
        #model.add(Dropout(dropout))

    model.add(Dense(1, activation="linear"))
    # compile the model
    model.compile(
        loss= loss,
        #metrics=[ "max_loss"],
        optimizer = tf.keras.optimizers.Adamax())
    # return compiled model
    return model

In [ ]:
"""Custom loss function to reduce the absolute error of the individual sample as well as the 
maximum error"""
def customLoss(y_true, y_pred):
    weight = 1.5
    difference = tf.abs(y_true - y_pred)
    exponent = tf.exp(tf.multiply(weight, difference))
    weighted_muls = tf.multiply(difference, exponent)
    boltzmann_op = tf.reduce_sum(weighted_muls) / tf.reduce_sum(exponent)
    loss = tf.add(boltzmann_op, tf.losses.mean_absolute_error(y_true, y_pred))

    return loss

In [ ]:
def preprocessData(x_train, x_test, y_train, y_test):
    x_train = OpenJsontoArr(x_train)
    x_test = OpenJsontoArr(x_test)
    y_train = OpenJsontoArr(y_train)
    y_test = OpenJsontoArr(y_test)
    y_train = y_train.flatten()
    y_test = y_test.flatten()

    df_y_train = pd.DataFrame({"y": y_train})
    df_weights_train = weightValues(df_y_train, weightMin=1, weightMax=2, namey="y", nameWeights="weights")
    df_y_test = pd.DataFrame({"y": y_test})
    df_weights_test = weightValues(df_y_test, weightMin=1, weightMax=2, namey="y", nameWeights="weights")

    y_test = np.array(df_weights_test)
    y_test = y_test[:, :2]
    y_train = np.array(df_weights_train)
    y_train = y_train[:, :2]

    return x_train, x_test, y_train, y_test

In [ ]:
def trainLSTM(x_train, x_test, y_train, y_test, window):
    #i = 0
    #summary = np.array(len(losses))
    #for loss in losses:
    lstmmodel = get_lstm_model(layers=[1], dropout=0, activation = "elu", input_shape = (window,6), loss = customLoss)

    summary = lstmmodel.fit(x_train, y_train[:,0], validation_data=(x_test, y_test), sample_weight = y_train[:,1], 
                            epochs = 2500, batch_size = 4000, callbacks=[earlyStop, stopNaN], shuffle = False, verbose = 0)

    y_pred_test = lstmmodel.predict(x_test)
    y_pred_train = lstmmodel.predict(x_train)

    diff_test = y_test[:, 0] - y_pred_test.flatten()
    error_avg_test = np.mean(abs(diff_test))
    error_max_test = max(abs(diff_test))

    diff_train = y_train[:, 0] - y_pred_train.flatten()
    error_avg_train = np.mean(abs(diff_train))
    error_max_train = max(abs(diff_train))

    #mean_absolute = tf.keras.losses.mean_absolute_error(y_cv[:,0], y_pred.flatten())
    #print("mean absolute: ", mean_absolute)
    print("Validation scores: ")
    print("max error: ", error_max_test)
    print("mean abs error: ", error_avg_test)
    print("Error over the training data: ")
    print("max error: ", error_max_train)
    print("mean abs error: ", error_avg_train)
    #i+=1
    
    return summary

In [ ]:
def plotTrainValLoss(summary):
    scatter_mode = "lines"
    train_loss = summary.history["loss"]
    validation_loss = summary.history["val_loss"]

    fig= make_subplots(rows=1, cols=1, shared_xaxes= True, print_grid= True)

    fig.add_trace(go.Scatter(y= train_loss, name= 'training loss', mode= scatter_mode), row= 1, col= 1)
    fig.add_trace(go.Scatter(y= validation_loss, name= 'validation loss', mode= scatter_mode), row= 1, col= 1)
    fig.update_yaxes(title_text= 'loss', row= 1, col= 1)

    fig.update_layout(height=600, width=1200, title_text="Training loss curve")
    fig.show()

In [ ]:
#losses = [customLoss()]
#,          tf.keras.losses.MeanSquaredError(),
#           tf.keras.losses.MeanAbsoluteError()]

earlyStop = EarlyStopping(monitor='val_loss', patience=100, verbose =1, mode = "auto")
#csvLogger = CSVLogger('X:\\KI Praktikum\\csvLoggerCustomLoss.xlsx')
stopNaN = TerminateOnNaN()

In [ ]:
x_train = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\training_data", "*_x.json"))
y_train = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\training_data", "*_y.json"))
x_test = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\validation_data", "*_x.json"))
y_test = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\validation_data", "*_y.json"))
window_size = [50, 60, 70, 50, 60, 70, 50, 60, 70]

In [ ]:
for i in range(len(x_train)):
    print(x_train[i])
    x_traini, x_testi, y_traini, y_testi = preprocessData(x_train[i], x_test[i], y_train[i], y_test[i])
    summary = trainLSTM(x_traini, x_testi, y_traini, y_testi, window_size[i])
    plotTrainValLoss(summary)

In [ ]:
# df_y_train = pd.DataFrame({"y": y_train})
# print(df_y_train.head(5))
# df_weights_train = weightValues(df_y_train, weightMin=1, weightMax=2, namey="y", nameWeights="weights")

# df_y_test = pd.DataFrame({"y": y_test})
# print(df_y_test.head(5))
# df_weights_test = weightValues(df_y_test, weightMin=1, weightMax=2, namey="y", nameWeights="weights")

In [ ]:
# #df_weights = df_weights.drop("0", axis=1)
# #print(df_weights.head(10))
# # x_weighted = np.zeros((x_cv.shape[0], x_cv.shape[1], x_cv.shape[2]+1))
# # x_weighted[:,:,:6] = x_cv
# # x_weighted[:,:,-1] = df_weights
# y_test = np.array(df_weights_test)
# y_test = y_test[:, :2]
# y_train = np.array(df_weights_train)
# y_train = y_train[:, :2]

In [ ]:
# modelPath = "X:\\KI Praktikum\\model\\23_09_01lstm_CustomLoss_max12_8.h5"
# lstmmodel.save(modelPath)